In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

### **Data Reading**

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@datalakeadbete.dfs.core.windows.net/customers")

df.display()

In [0]:
df = df.drop("_rescued_data")

df.display()

In [0]:
df = df.withColumn("domains", split(col("email"), "@") [1])

df.display()

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).orderBy(desc("total_customers")).display()

In [0]:
df_gmail = df.filter(col('domains') == "gmail.com")
df_gmail.display()

df_hotmail = df.filter(col('domains') == "hotmail.com")
df_hotmail.display()

df_yahoo = df.filter(col('domains') == "yahoo.com")
df_yahoo.display()

In [0]:
df = df.withColumn("full_name", concat(col("first_name"), lit(" "), col("last_name")))

df = df.drop("first_name", "last_name")

df.display()

### **Data Writing**

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@datalakeadbete.dfs.core.windows.net/customers")

### **Create Schema and Table**

In [0]:
%sql
create schema if not exists`databricks-catas`.silver

In [0]:
%sql
create table if not exists `databricks-catas`.silver.Customers_silver
using delta
LOCATION 'abfss://silver@datalakeadbete.dfs.core.windows.net/customers'

In [0]:
%sql
select * from `databricks-catas`.silver.Customers_silver